### Hints for homework with the commands2 library - 2021 1231 CJH
---

### What is already in the template
* The robotpy example (hatch) has a working drivetrain (drivesusbsystem.py) and a drivetrain sim
* It has a robotcontainer.py that can show you how to 
  * import and instantiate subsystems
  * bind commands to joystick buttons
* The template already has several commands written in the comands folder
  * look through them and understand what they do
  * use these as templates to help you make your own

### #1: Driving for a set amount of time

General hints:
* make a new command.  the easiest way is to copy one of the simple ones and rename it
* you can use the command drivedistance.py for help
  * that one drives a distance, so it is checking the encoder to see if it should end
  * if you just want to drive for a time, you can set the command timeout and check for timed out in the isFinished() part of the command
  * they took away the isTimedOut and setTimeOut functions in commands2
  * so you should get the time when you initialize and see if the difference is greater than the time you wanted to drive
 

In [7]:
import wpilib
import commands2

In [8]:
drive_time = 2 # for instance, assign to self.drive time in init
# in initialize stage of command
start_time = wpilib.Timer.getMatchTime()
# in the isFinished stage of the command
elapsed_time = wpilib.Timer.getMatchTime() - start_time
if elapsed_time > drive_time:
    return True
else:
    return False

SyntaxError: 'return' outside function (<ipython-input-8-5e98692f212e>, line 7)

In [6]:
start_time

0.0